In [34]:
import pandas as pd
import numpy as np
import sklearn
import re
import xgboost as xgb
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt

from sklearn import linear_model
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 500)

### Import Files

In [11]:
housing_data = pd.read_csv('data/2014_Housing_Market_Analysis_Data_by_Zip_Code.csv')
crime_2016_data = pd.read_csv('data/2016_Annual_Crime_Data.csv')
crime_2015_data = pd.read_csv('data/Annual_Crime_Dataset_2015.csv')
library_data = pd.read_csv('data/Austin_Public_Library_Locations.csv')
water_consumption_data = pd.read_csv('data/Austin_Water_-_Residential_Water_Consumption.csv')
campaign_finance_data = pd.read_csv('data/Campaign_Finance_Data_-_Report_Detail_Dataset.csv')
park_data = pd.read_csv('data/City_of_Austin_Parks_data.csv')
public_art_data = pd.read_csv('data/City_of_Austin_Public_Art_Collection.csv')
public_venue_data = pd.read_csv('data/Creative_Workspaces__Performance_Venues__Galleries___Museums.csv')
ev_charging_data = pd.read_csv('data/Electric_Vehicle_Charging_Network.csv')
restaurant_inspection_data = pd.read_csv('data/Restaurant_Inspection_Scores.csv')
traffic_camera_data = pd.read_csv('data/Traffic_Cameras.csv')

### Extract Zip Code from Address column in library data

In [12]:
library_data['Zip_Code'] = library_data['Address'].str.findall('\s+\d+\n')
library_data['Zip_Code'] = library_data['Zip_Code'].str[0].str[:6]

### Clean Restaurant Inspections (Multiple Dates per Restaurant)

In [13]:
restaurant_max_inspection = restaurant_inspection_data.groupby('Restaurant Name', as_index = False)['Inspection Date'].agg('max')

restaurant_inspection = pd.merge(left = restaurant_max_inspection, right = restaurant_inspection_data
                      , how = 'inner'
                      , left_on = ['Restaurant Name', 'Inspection Date']
                      , right_on = ['Restaurant Name', 'Inspection Date'])

restaurant_inspection_data = restaurant_inspection.drop_duplicates()

restaurant_inspection_data['Zip Code'] = restaurant_inspection_data['Zip Code'].str[-5:]

/home/tobiasfunke/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


### Aggregate Files to Zip Code level

In [14]:
crime_2015 = crime_2015_data.groupby('GO Location Zip', as_index = False).size()
crime_2015 = crime_2015.reset_index()
crime_2015 = crime_2015.rename(columns = {0: 'crimes_2015'})
# print(crime_2015.head())

crime_2016 = crime_2016_data.groupby('GO Location Zip', as_index = False).size()
crime_2016 = crime_2016.reset_index()
crime_2016 = crime_2016.rename(columns = {0: 'crimes_2016'})
# crime_2016.head()

ev_charging = pd.DataFrame(ev_charging_data.groupby('Postal Code', as_index = False).size())
ev_charging = ev_charging.reset_index()
ev_charging = ev_charging.rename(columns = {0: 'ev_charging_stations'})
# ev_charging.head()

restaurant_inspection = restaurant_inspection_data.groupby('Zip Code', as_index = False)['Score'].agg(['median','size'])
restaurant_inspection = restaurant_inspection.reset_index()
restaurant_inspection = restaurant_inspection.rename(columns = {'median': 'median_rest_insp_score', 'size':'number_of_inspections'})
restaurant_inspection['Zip Code'] = pd.to_numeric(restaurant_inspection['Zip Code'])
# print(restaurant_inspection.head())

public_art = public_art_data.groupby('Location Zip Code', as_index = False).size()
public_art = public_art.reset_index()
public_art = public_art.rename(columns = {0: 'public_art_installations'})
# public_art.head()

public_venue = public_venue_data.groupby('ZIP', as_index = False).size()
public_venue = public_venue.reset_index()
public_venue = public_venue.rename(columns = {0: 'public_venues'})
# public_venue.head()

park = park_data.groupby('ZIP_CODE', as_index = False).size()
park = park.reset_index()
park = park.rename(columns = {0: 'parks'})
# park.head()

water_consumption = water_consumption_data.groupby('Postal Code', as_index = False)['Total Gallons'].median()
water_consumption = water_consumption.reset_index()
water_consumption = water_consumption.rename(columns = {'Total Gallons': 'median_water_used_gal'})
del water_consumption['index']
# water_consumption.head()

library = library_data.groupby('Zip_Code', as_index = False).size()
library = library.reset_index()
library = library.rename(columns = {0: 'libraries'})
library['Zip_Code'] = pd.to_numeric(library['Zip_Code'].str.strip())

### Combine Files

In [86]:
combined_data = crime_2015.copy()
# print(crime_2015.shape)
print(combined_data.shape)

combined_data = pd.merge(left = combined_data, right = crime_2016
                      , how = 'inner'
                      , on = 'GO Location Zip')
# print(combined_data.shape)

combined_data = pd.merge(left = combined_data, right = ev_charging
                      , how = 'left'
                      , left_on = 'GO Location Zip'
                      , right_on = 'Postal Code')
del combined_data['Postal Code']
# print(combined_data.shape)

combined_data = pd.merge(left = combined_data, right = housing_data
                      , how = 'inner'
                      , left_on = 'GO Location Zip'
                      , right_on = 'Zip Code')
del combined_data['Zip Code']
# print(combined_data.shape)

combined_data = pd.merge(left = combined_data, right = park
                      , how = 'left'
                      , left_on = 'GO Location Zip'
                      , right_on = 'ZIP_CODE')
del combined_data['ZIP_CODE']
# print(combined_data.shape)

combined_data = pd.merge(left = combined_data, right = public_art
                      , how = 'left'
                      , left_on = 'GO Location Zip'
                      , right_on = 'Location Zip Code')
del combined_data['Location Zip Code']
# print(combined_data.shape)

combined_data = pd.merge(left = combined_data, right = water_consumption
                      , how = 'left'
                      , left_on = 'GO Location Zip'
                      , right_on = 'Postal Code')
del combined_data['Postal Code']
# print(combined_data.shape)

combined_data = pd.merge(left = combined_data, right = library
                      , how = 'left'
                      , left_on = 'GO Location Zip'
                      , right_on = 'Zip_Code')
del combined_data['Zip_Code']
# print(combined_data.shape)
# print(combined_data)

combined_data.head()

(47, 2)


,GO Location Zip,crimes_2015,crimes_2016,ev_charging_stations,Population below poverty level,Median household income,"Non-White, Non-Hispanic or Latino","Hispanic or Latino, of any race",Population with disability,Unemployment,Large households (5+ members),"Homes affordable to people earning less than $50,000","Rentals affordable to people earning less than $25,000",Rent-restricted units,Housing Choice Voucher holders,Median rent,Median home value,Percentage of rental units in poor condition,"Percent change in number of housing units, 2000-2012",Owner units affordable to average retail/service worker,Rental units affordable to average retail/service worker,Rental units affordable to average artist,Owner units affordable to average artist,Rental units affordable to average teacher,Owner units affordable to average teacher,Rental units affordable to average tech worker,Owner units affordable to average tech worker,"Change in percentage of population below poverty, 2000-2012","Change in median rent, 2000-2012","Change in median home value, 2000-2012",Percentage of homes within 1/4-mi of transit stop,Average monthly transportation cost,Percentage of housing and transportation costs that is transportation-related,parks,public_art_installations,median_water_used_gal,libraries
0,78617,276,285,NaN,18%,$43957,12%,67%,10%,15%,23%,NaN,11%,1%,8%,$1041,$100600,0.4%,34%,NaN,7%,24%,NaN,63%,NaN,99%,NaN,101%,74%,21%,16%,$865,42%,5.0,NaN,3367450,NaN
1,78701,2103,2076,50.0,20%,$68152,16%,14%,10%,9%,0%,7%,7%,9%,0%,$1590,$338300,0.8%,134%,0%,7%,12%,1%,29%,7%,90%,30%,12%,115%,59%,97%,$433,23%,13.0,114.0,942600,3.0
2,78702,1668,1582,18.0,33%,$34734,18%,56%,14%,11%,10%,21%,41%,39%,2%,$766,$175400,2.2%,15%,0%,39%,51%,2%,80%,15%,99%,67%,3%,73%,207%,96%,$590,39%,22.0,85.0,2884950,4.0
3,78703,738,660,10.0,10%,$92606,9%,9%,6%,4%,4%,3%,11%,0%,0%,$1183,$621900,0.4%,1%,0%,8%,25%,0%,51%,2%,92%,13%,7%,65%,104%,67%,$629,25%,18.0,8.0,4255050,1.0
4,78704,2571,2557,29.0,21%,$50248,7%,30%,9%,7%,3%,13%,12%,13%,1%,$940,$338200,0.4%,9%,0%,11%,26%,2%,76%,13%,99%,33%,33%,40%,126%,76%,$629,33%,26.0,14.0,13373950,1.0


### Remove String Characters

In [87]:
combined_data['Population below poverty level'] = pd.to_numeric(combined_data['Population below poverty level'].str.replace('\%', ''))
combined_data['Median household income'] = pd.to_numeric(combined_data['Median household income'].str.replace('\$', ''))
combined_data['Non-White, Non-Hispanic or Latino'] = pd.to_numeric(combined_data['Non-White, Non-Hispanic or Latino'].str.replace('\%', ''))
combined_data['Hispanic or Latino, of any race'] = pd.to_numeric(combined_data['Hispanic or Latino, of any race'].str.replace('\%', ''))
combined_data['Population with disability'] = pd.to_numeric(combined_data['Population with disability'].str.replace('\%', ''))
combined_data['Unemployment'] = pd.to_numeric(combined_data['Unemployment'].str.replace('\%', ''))
combined_data['Change in percentage of population below poverty, 2000-2012'] = pd.to_numeric(combined_data['Change in percentage of population below poverty, 2000-2012'].str.replace('\%', ''))
combined_data['Change in median rent, 2000-2012'] = pd.to_numeric(combined_data['Change in median rent, 2000-2012'].str.replace('\%', ''))
combined_data['Change in median home value, 2000-2012'] = pd.to_numeric(combined_data['Change in median home value, 2000-2012'].str.replace('\%', ''))
combined_data['Percentage of homes within 1/4-mi of transit stop'] = pd.to_numeric(combined_data['Percentage of homes within 1/4-mi of transit stop'].str.replace('\%', ''))
combined_data['Average monthly transportation cost'] = pd.to_numeric(combined_data['Average monthly transportation cost'].str.replace('\$', ''))
combined_data['Percentage of housing and transportation costs that is transportation-related'] = pd.to_numeric(combined_data['Percentage of housing and transportation costs that is transportation-related'].str.replace('\%', ''))
combined_data['Large households (5+ members)'] = pd.to_numeric(combined_data['Large households (5+ members)'].str.replace('\%', ''))
combined_data['Homes affordable to people earning less than $50,000'] = pd.to_numeric(combined_data['Homes affordable to people earning less than $50,000'].str.replace('\%', ''))
combined_data['Rentals affordable to people earning less than $25,000'] = pd.to_numeric(combined_data['Rentals affordable to people earning less than $25,000'].str.replace('\%', ''))
combined_data['Rent-restricted units'] = pd.to_numeric(combined_data['Rent-restricted units'].str.replace('\%', ''))
combined_data['Median rent'] = pd.to_numeric(combined_data['Median rent'].str.replace('\$', ''))
combined_data['Median home value'] = pd.to_numeric(combined_data['Median home value'].str.replace('\$', ''))
combined_data['Percentage of rental units in poor condition'] = pd.to_numeric(combined_data['Percentage of rental units in poor condition'].str.replace('\%', ''))
combined_data['Housing Choice Voucher holders'] = pd.to_numeric(combined_data['Housing Choice Voucher holders'].str.replace('\%', ''))
combined_data['Percent change in number of housing units, 2000-2012'] = pd.to_numeric(combined_data['Percent change in number of housing units, 2000-2012'].str.replace('\%', ''))
combined_data['Owner units affordable to average retail/service worker'] = pd.to_numeric(combined_data['Owner units affordable to average retail/service worker'].str.replace('\%', ''))
combined_data['Rental units affordable to average retail/service worker'] = pd.to_numeric(combined_data['Rental units affordable to average retail/service worker'].str.replace('\%', ''))
combined_data['Rental units affordable to average artist'] = pd.to_numeric(combined_data['Rental units affordable to average artist'].str.replace('\%', ''))
combined_data['Owner units affordable to average artist'] = pd.to_numeric(combined_data['Owner units affordable to average artist'].str.replace('\%', ''))
combined_data['Rental units affordable to average teacher'] = pd.to_numeric(combined_data['Rental units affordable to average teacher'].str.replace('\%', ''))
combined_data['Owner units affordable to average teacher'] = pd.to_numeric(combined_data['Owner units affordable to average teacher'].str.replace('\%', ''))
combined_data['Rental units affordable to average tech worker'] = pd.to_numeric(combined_data['Rental units affordable to average tech worker'].str.replace('\%', ''))
combined_data['Owner units affordable to average tech worker'] = pd.to_numeric(combined_data['Owner units affordable to average tech worker'].str.replace('\%', ''))

### Rename Columns (make column selection easier below

In [88]:
combined_data = combined_data.rename(columns = {'GO Location Zip': 'zip'
                                                , 'Population below poverty level': 'pop_below_poverty_level'
                                                , 'Median household income': 'median_household_income'
                                                , 'Non-White, Non-Hispanic or Latino': 'pop_nonwhite_nonhispanic'
                                                ,  'Hispanic or Latino, of any race': 'pop_hispanic'
                                                , 'Population with disability': 'pop_disabled'
                                                , 'Large households (5+ members)': 'pop_large_household'
                                                , 'Homes affordable to people earning less than $50,000': 'homes_affordable_for_ppl_earning_LT_50K'
                                                , 'Rentals affordable to people earning less than $25,000': 'rentals_affordable_for_ppl_earning_LT_25K'
                                                , 'Rent-restricted units': 'rent_restricted_units'
                                                , 'Housing Choice Voucher holders': 'housing_choice_voucher_holders'
                                                , 'Median rent': 'median_rent'
                                                , 'Median home value': 'median_home_value'
                                                , 'Percentage of rental units in poor condition': 'perc_rentals_in_poor_cond'
                                                , 'Percent change in number of housing units, 2000-2012': 'perc_change_in_units_2000_2012'
                                                , 'Owner units affordable to average retail/service worker': 'homes_affordable_to_retail_service_worker'
                                                , 'Rental units affordable to average retail/service worker': 'rentals_affordable_to_retail_service_workers'
                                                , 'Rental units affordable to average artist': 'rentals_affordable_to_artists'
                                                , 'Owner units affordable to average artist': 'homes_affordable_to_artists'
                                                , 'Rental units affordable to average teacher': 'rentals_affordable_to_teachers'
                                                , 'Owner units affordable to average teacher': 'homes_affordable_to_teachers'
                                                , 'Rental units affordable to average tech worker': 'rentals_affordable_to_tech_workers'
                                                , 'Owner units affordable to average tech worker': 'homes_affordable_to_tech_workers'
                                                , 'Change in percentage of population below poverty, 2000-2012': 'perc_change_in_pop_2000_2012'
                                                , 'Change in median rent, 2000-2012': 'perc_change_median_rent_2000_2012'
                                                , 'Change in median home value, 2000-2012': 'perc_change_median_home_2000_2012'
                                                , 'Percentage of homes within 1/4-mi of transit stop': 'perc_homes_win_quartermile_transit_stop'
                                                , 'Average monthly transportation cost': 'avg_monthly_transportation_cost'
                                                , 'Percentage of housing and transportation costs that is transportation-related': 'perc_cost_related_to_transportation'
                                               })

combined_data.columns

Index(['zip', 'crimes_2015', 'crimes_2016', 'ev_charging_stations',
       'pop_below_poverty_level', 'median_household_income',
       'pop_nonwhite_nonhispanic', 'pop_hispanic', 'pop_disabled',
       'Unemployment', 'pop_large_household',
       'homes_affordable_for_ppl_earning_LT_50K',
       'rentals_affordable_for_ppl_earning_LT_25K', 'rent_restricted_units',
       'housing_choice_voucher_holders', 'median_rent', 'median_home_value',
       'perc_rentals_in_poor_cond', 'perc_change_in_units_2000_2012',
       'homes_affordable_to_retail_service_worker',
       'rentals_affordable_to_retail_service_workers',
       'rentals_affordable_to_artists', 'homes_affordable_to_artists',
       'rentals_affordable_to_teachers', 'homes_affordable_to_teachers',
       'rentals_affordable_to_tech_workers',
       'homes_affordable_to_tech_workers', 'perc_change_in_pop_2000_2012',
       'perc_change_median_rent_2000_2012',
       'perc_change_median_home_2000_2012',
       'perc_homes_win_qu

### Create Training and Validation Sets

In [54]:
train, test = train_test_split(combined_data, test_size = 0.2)

print(train.shape)
print(test.shape)

(28, 37)
(8, 37)


### Linear Models

In [97]:
# http://www.statsmodels.org/stable/index.html


linear_model1 = sm.ols('crimes_2015 ~ parks + ev_charging_stations + pop_below_poverty_level + median_rent\
                                    + pop_hispanic'
                       , data = train).fit()
linear_model1.summary() #.486


linear_model2 = sm.ols('crimes_2015 ~ ev_charging_stations + pop_hispanic + median_household_income\
                                    + median_water_used_gal'
                       , data = train).fit()
linear_model2.summary() #.589


linear_model3 = sm.ols('crimes_2015 ~ Unemployment + perc_change_in_pop_2000_2012 + public_art_installations\
                                    + pop_below_poverty_level + homes_affordable_to_tech_workers'
                       , data = train).fit()
linear_model3.summary() #.091

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            crimes_2015   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     1.422
Date:                Fri, 11 May 2018   Prob (F-statistic):              0.269
Time:                        21:18:17   Log-Likelihood:                -176.45
No. Observations:                  22   AIC:                             364.9
Df Residuals:                      16   BIC:                             371.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                          748.6823    610.343      1.227      0.238    -545.188    2042.553
Unemployment                       -55.0664    123.487     -0.446      0.662    -316.848     206.715
perc_change_in_pop_2000_2012        -2.7875      2.890     -0.965      0.349      -8.913       3.338
public_art_installations             9.8578      7.299      1.351      0.196      -5.615      25.331
pop_below_poverty_level             -1.0075     22.779     -0.044      0.965     -49.297      47.282
homes_affordable_to_tech_workers    18.7983      9.262      2.030      0.059      -0.836      38.432
==============================================================================
Omnibus:                        2.549   Durbin-Watson:                   2.660
Prob(Omnibus):                  0.280   Jarque-Bera (JB):                1.437
Skew:                           0.620   Prob(JB):                        0.487
Kurtosis:                       3.174   Cond. No.                         434.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Make Predictions

In [131]:
predictions = pd.DataFrame(linear_model2.predict(test))

test_with_predictions = pd.merge(test, pd.DataFrame(predictions), left_index = True, right_index = True, how = 'inner')
test_with_predictions = test_with_predictions.rename(columns = {0: 'predictions'})
test_with_predictions = test_with_predictions[['zip', 'crimes_2015', 'predictions']]
test_with_predictions['diff_sq'] = np.power(test_with_predictions['crimes_2015'] - test_with_predictions['predictions'], 2)

np.sqrt(np.mean(test_with_predictions['diff_sq']))

732.4372837959752

In [17]:
'''
Forward Selection: Correlation with ZipCode and Crimes_2015 & crimes_2016

1) Population below poverty level
2) Median household income
3) Non-White, Non-Hispanic or Latino
4) Hispanic or Latino, of any race
5) Unemployment
6) Population with disability
7) Large households (5+ members)
8) 'parks'
9) 'Average monthly transportation cost'
10) 'Percentage of housing and transportation costs that is transportation-related'
11) 'Percentage of homes within 1/4-mi of transit stop'
12) 'Percentage of rental units in poor condition'
13) 'Median rent'
14) 'Median home value'
15) 'Rentals affordable to people earning less than $25,000'

'''

# The x has the ZipCode and all other relating attributes to it.
x = combined_data[['GO Location Zip',
                   'Population below poverty level',
                   'Non-White, Non-Hispanic or Latino',
                   'Unemployment',
                   'Hispanic or Latino, of any race',
                   'Population with disability',
                   'Large households (5+ members)',
                   'Percentage of housing and transportation costs that is transportation-related',
# After Forward Selection the excess ones
                   
#These are problomatic and need suggestions to incorporte them                   
#                  'parks',
#                  'Average monthly transportation cost',
#                  'Median household income',                   
#                  'Percentage of homes within 1/4-mi of transit stop',
#                  'Percentage of rental units in poor condition',
#                  'Median rent',
#                  'Median home value',
#                  'Rentals affordable to people earning less than $25,000'
                  ]]

#This is used as the target to compare the values. 
y = combined_data[['GO Location Zip', 'crimes_2015']]
z = combined_data[['GO Location Zip', 'crimes_2016']]

# plt.scatter(x, y)
# plt.show()

In [18]:
reg = linear_model.LinearRegression()
reg.fit(x, y)
a = reg.predict(x)
np.mean((a-y)**2)

GO Location Zip         0.000000
crimes_2015        501762.443149
dtype: float64

In [19]:
reg.fit(x, z)
a = reg.predict(x)
np.mean((a-z)**2)

GO Location Zip         0.000000
crimes_2016        466540.654538
dtype: float64